In [1]:
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.interfaces.elegant import elegant_to_data
from h5py import File


import numpy as np

from pytao import Tao

import subprocess
import os

root_dir = os.getcwd()

## Helper functions

In [2]:
# Make sure root_dir/code has been added to $PTHONPATH
from parse_output import get_twiss_tao, get_twiss_elegant
from write_lattice import write_bmad_lattice, write_elegant_lattice

# Prepare Bmad and elegant lattice files

### Refer to Bmad and Elegant manuals for valid element parameters

In [3]:
quad_dict_bmad={'L':0.03, 'k1':2.0, 'x_offset':1E-4}

In [5]:
quad_dict_elegant={'L':0.03, 'K1':2.0, 'dx':1E-4}

In [4]:
write_bmad_lattice('quad', quad_dict_bmad, root_dir)

In [6]:
write_elegant_lattice("QUAD", quad_dict_elegant, root_dir)

# run Tao (bmad)

In [7]:
# This line can only be run ONCE
tao = Tao(f'-init {root_dir}/bmad/tao.init -noplot')

In [8]:
tao.cmd('beamon')
tao.cmd('write beam ./bmad/beam_end_tao.hdf5 -at END');  # For analysis

#### collect Bmad beam and Twiss

In [9]:
H5FILE = root_dir+'/bmad/beam_end_tao.hdf5'
Ptao = ParticleGroup(H5FILE)

twiss_tao = get_twiss_tao(tao, 'END')

# run Elegant

In [10]:
# Make sure elegant has been installed and located
!which elegant

/Users/wlou/Code/elegant/bin/elegant


In [11]:
os.chdir(root_dir+'/elegant')
subprocess.run(["elegant", "one_ele.ele"], stdout=subprocess.DEVNULL)
#os.chdir(root_dir)

CompletedProcess(args=['elegant', 'one_ele.ele'], returncode=0)

#### collect Elegant beam and Twiss

In [12]:
OUTFILE = root_dir+'/elegant/BPM_END.out'
DAT = elegant_to_data(OUTFILE, charge = 13e-12)
Pele = ParticleGroup(data=DAT)

elegant_twiss_file_name = root_dir + '/elegant/one_ele.twi'
twiss_elegant = get_twiss_elegant(elegant_twiss_file_name)

# Compare beam 6D coordinates and 8 Twiss

In [13]:
twiss_tao

array([ 9.92226266e+00,  1.58968962e+00,  9.95811888e+00,  3.96874699e-01,
       -8.99730024e-08, -5.99640049e-06,  0.00000000e+00,  0.00000000e+00])

In [15]:
twiss_elegant - twiss_tao

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.55111512e-17,
        0.00000000e+00, -8.47032947e-22,  0.00000000e+00,  0.00000000e+00])

In [33]:
def compare_twiss(twiss_tao, twiss_elegant, tol=1e-6):
    print("Agree in all 8 Twiss...?", np.all(abs(twiss_tao - twiss_elegant) < tol))
    return None

In [34]:
compare_twiss(twiss_tao, twiss_elegant, 1e-6)

Agree in all 8 Twiss...? True


In [18]:
Ptao.x

array([ 8.99865008e-08, -9.99010148e-04,  9.99190121e-04, -2.99010143e-05,
        3.00809873e-05,  8.99865008e-08,  8.99865008e-08,  8.99865008e-08,
        8.99865008e-08,  8.99865008e-08,  8.99865008e-08,  9.00765639e-08,
        8.98966177e-08])

In [20]:
Ptao.x - Pele.x

array([ 1.93679966e-14,  2.13048112e-13, -1.74312171e-13,  1.50168054e-11,
       -1.49780694e-11,  1.93679966e-14,  1.93679966e-14,  1.93679966e-14,
        1.93679966e-14,  1.93679966e-14,  1.93679966e-14,  1.09398184e-13,
        1.09257031e-13])

In [16]:
def compare_two_P(Ptao, Pele, tol=1e-9):
    
    assert Ptao.n_particle == Pele.n_particle, "The two particle groups need to have the same number of particles"
    
    print(f"Comparing {Ptao.n_particle} particles...")
    
    print("Agree in all x...?", np.all(abs(Ptao.x - Pele.x) < tol))
    print("Agree in all xp...?", np.all(abs(Ptao.xp - Pele.xp) < tol))
    print("Agree in all y...?", np.all(abs(Ptao.y - Pele.y) < tol))
    print("Agree in all yp...?", np.all(abs(Ptao.yp - Pele.yp) < tol))
    print("Agree in all t...?", np.all(abs(Ptao.t - Pele.t) < tol))
    
    print("Agree in all delta (E0=10MeV)?", np.all(abs(Ptao.energy - Pele.energy)/10E6 < tol))
    

In [17]:
compare_two_P(Ptao, Pele, tol=1e-9)

Comparing 13 particles...
Agree in all x...? True
Agree in all xp...? True
Agree in all y...? True
Agree in all yp...? True
Agree in all t...? True
Agree in all delta (E0=10MeV)? True
